In [62]:
import os
import sqlite3
import pandas as pd
import tqdm
import pickle

## Text_title

In [63]:
db_name = '[Full_TFIDF]Credibility_research_20180906.db' #DB 파일명
# db 생성
con = sqlite3.connect( db_name )
cur = con.cursor()

In [64]:
sql = '''
select p.Post_id,p.Category,p.Title,p.Text,u.Credibility,group_concat(img.Img_id) as img_count
    FROM Post as p
    Left JOIN user as u
        ON p.User_id = u.User_id
    Left JOIN Img as img
        ON p.Post_id = img.Post_id
    GROUP BY p.Post_id;
    '''
cur.execute(sql)
rows = cur.fetchall()

In [65]:
col_name = [i[0] for i in cur.description]

In [66]:
test_df = pd.DataFrame(rows)

In [67]:
test_df.columns = col_name

In [68]:
test_df.columns.tolist()

['Post_id', 'Category', 'Title', 'Text', 'Credibility', 'img_count']

In [69]:
test_df = test_df[test_df['Category'] != "무작위"].iloc[:16304]

In [70]:
Title_Text_df = test_df[['Post_id','Text','Title']]

In [71]:
Title_Text_df = Title_Text_df.dropna()
Title_Text_df = Title_Text_df.reset_index(drop=True)

In [72]:
Title_Text_df['Post_id'] = Title_Text_df['Post_id'].apply(lambda x : int(x))

## Best_feature

In [73]:
df_polarity = pd.read_csv('polarity_socre.csv',sep='\t',encoding='utf-8')
df = pd.read_csv('[Final]Best_feautres_df.csv',sep='\t',encoding='utf-8')
df1 = pd.read_csv('additional_feature_from_user_study.csv',sep='\t',encoding='utf-8')
df2 = pd.read_csv('Count_mistake.csv',sep='\t',encoding='utf-8')

In [74]:
Total_df = pd.merge(df,df1)

In [75]:
Total_df = pd.merge(Total_df,df2)

In [76]:
drop_column_list = ['Blog_mobile_profile_img','Count_neighbors','Post_interval_average',
 'Text_similarity',
 'Total_post',
 'User_Id_len',
 'Comment_interaction',
 'Blog_mobile_profile_img',
 'Opening_date','Blog_info_text_len']

In [77]:
new_df = Total_df.drop(drop_column_list,axis=1)

In [78]:
new_df['Polarity'] = df_polarity['polarity'].tolist()

In [79]:
Analysis_df = new_df.fillna(0)

## Text_feature, Title_feature vectorizer 

In [80]:
Text = Title_Text_df['Text']
Title = Title_Text_df['Title']

### Text

In [81]:
vectorizer_Text = pickle.load(open("tfidf_text.pkl", "rb"))

In [82]:
tfidf_text = vectorizer_Text.transform(Text)

In [23]:
Text_df = pd.DataFrame(tfidf_text.toarray())

### Title

In [26]:
vectorizer_Title = pickle.load(open("tfidf_title_.pkl", "rb"))

In [27]:
tfidf_Title = vectorizer_Title.transform(Title)

In [28]:
Title_df = pd.DataFrame(tfidf_Title.toarray())

### Post_id_Text_Title merge

In [83]:
db_post_id = Title_Text_df[['Post_id']]

In [84]:
db_df = pd.concat([db_post_id,Text_df],axis=1)

## Best_feature

In [85]:
df_polarity = pd.read_csv('polarity_socre.csv',sep='\t',encoding='utf-8')
df = pd.read_csv('[Final]Best_feautres_df.csv',sep='\t',encoding='utf-8')
df1 = pd.read_csv('additional_feature_from_user_study.csv',sep='\t',encoding='utf-8')
df2 = pd.read_csv('Count_mistake.csv',sep='\t',encoding='utf-8')

In [86]:
Total_df = pd.merge(df,df1)

In [87]:
Total_df = pd.merge(Total_df,df2)

In [88]:
drop_column_list = ['Blog_mobile_profile_img','Count_neighbors','Post_interval_average',
 'Text_similarity',
 'Total_post',
 'User_Id_len',
 'Comment_interaction',
 'Blog_mobile_profile_img',
 'Opening_date','Blog_info_text_len']

In [89]:
new_df = Total_df.drop(drop_column_list,axis=1)

In [90]:
new_df['Polarity'] = df_polarity['polarity'].tolist()

In [91]:
Analysis_df = new_df.fillna(0)

In [92]:
Final_df = pd.merge(db_df,Analysis_df,how='inner',on=['Post_id'])

In [93]:
from sklearn.utils import shuffle
Final_df = shuffle(Final_df)

In [94]:
x = Analysis_df.drop(columns=['Post_id','Credibility'],axis=1)
y = Analysis_df[['Credibility']]
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(x) 

# Min-max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
scaler.fit(x) 
X_scaled = pd.DataFrame(scaler.transform(x),columns = x.columns)

In [95]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_scaled , y , test_size=0.33, random_state=42)

In [96]:
import time
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [97]:
mlp_clf = MLPClassifier(
    hidden_layer_sizes=(5,),
    max_iter=100,
    alpha=1e-4,
    solver='sgd',
    verbose=10,
    tol=1e-4,
    random_state=1,
    learning_rate_init=.1
)

In [98]:
start = time.time()
mlp_clf.fit(x_train, y_train)
end = time.time()
print('Time: {:f}s'.format(end-start))

/Users/yonggeol/miniconda3/envs/py/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.57970963
Iteration 2, loss = 0.51538698
Iteration 3, loss = 0.50458510
Iteration 4, loss = 0.49774990
Iteration 5, loss = 0.49145755
Iteration 6, loss = 0.48628828
Iteration 7, loss = 0.48225134
Iteration 8, loss = 0.47853119
Iteration 9, loss = 0.47497491
Iteration 10, loss = 0.47419778
Iteration 11, loss = 0.47243502
Iteration 12, loss = 0.46793233
Iteration 13, loss = 0.46546342
Iteration 14, loss = 0.46582010
Iteration 15, loss = 0.46438281
Iteration 16, loss = 0.46280967
Iteration 17, loss = 0.46123030
Iteration 18, loss = 0.45862092
Iteration 19, loss = 0.45939017
Iteration 20, loss = 0.45779517
Iteration 21, loss = 0.45566898
Iteration 22, loss = 0.45413932
Iteration 23, loss = 0.45220529
Iteration 24, loss = 0.45053480
Iteration 25, loss = 0.44873120
Iteration 26, loss = 0.44774205
Iteration 27, loss = 0.44462749
Iteration 28, loss = 0.44196618
Iteration 29, loss = 0.43773364
Iteration 30, loss = 0.43375885
Iteration 31, loss = 0.43426158
Iteration 32, los

In [99]:
y_pred = mlp_clf.predict(x_test)
print("테스트 정확도: {:.3f}".format(accuracy_score(y_pred, y_test)))

테스트 정확도: 0.818


In [100]:
from sklearn.metrics import classification_report
print(classification_report(y_test, mlp_clf.predict(x_test)))

             precision    recall  f1-score   support

          0       0.72      0.88      0.79      2088
          1       0.91      0.78      0.84      3293

avg / total       0.83      0.82      0.82      5381



In [101]:
x = Analysis_df.drop(columns=['Post_id','Credibility'],axis=1)
y = Analysis_df[['Credibility']]
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(x) 

# Min-max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
scaler.fit(x) 
X_scaled = pd.DataFrame(scaler.transform(x),columns = x.columns)

In [102]:
X_scaled['Post_id'] = Analysis_df['Post_id']

In [103]:
Best_df = pd.concat([X_scaled,y],axis=1)

In [44]:
Final_df = pd.merge(db_df,Best_df,how='inner',on=['Post_id'])

In [45]:
from sklearn.utils import shuffle
Final_df = shuffle(Final_df)

In [46]:
x = Analysis_df.drop(columns='Credibility',axis=1)
y = Analysis_df[['Credibility']]

In [47]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x , y , test_size=0.33, random_state=42)

In [48]:
import time
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [49]:
mlp_clf = MLPClassifier(
    hidden_layer_sizes=(5,),
    max_iter=100,
    alpha=1e-4,
    solver='sgd',
    verbose=10,
    tol=1e-4,
    random_state=1,
    learning_rate_init=.1
)

In [50]:
start = time.time()
mlp_clf.fit(x_train, y_train)
end = time.time()
print('Time: {:f}s'.format(end-start))

Iteration 1, loss = 1565124438221226.50000000
Iteration 2, loss = 2081742061425933.00000000
Iteration 3, loss = 2083687627272343.00000000
Iteration 4, loss = 2083578000517334.00000000
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Time: 0.056166s


/Users/yonggeol/miniconda3/envs/py/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [51]:
y_pred = mlp_clf.predict(x_test)
print("테스트 정확도: {:.3f}".format(accuracy_score(y_pred, y_test)))

테스트 정확도: 0.612


In [52]:
from sklearn.metrics import classification_report
print(classification_report(y_test, mlp_clf.predict(x_test)))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      2088
          1       0.61      1.00      0.76      3293

avg / total       0.37      0.61      0.46      5381



/Users/yonggeol/miniconda3/envs/py/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
